In [16]:
import pandas as pd
import sklearn
import re
df=pd.read_csv('movie_data.csv')

In [17]:
pd.set_option("max.colwidth",0)

In [18]:
text_col = df['review']
text_col = text_col.apply(lambda x: x.lower())
text_col = text_col.apply(lambda x: re.sub(r'\d+', '', x))
text_col = text_col.apply(lambda x: re.sub("[.]", " ", x))
text_col = text_col.apply(lambda x: re.sub(r'\W+', ' ', x))
text_col = text_col.apply(lambda x: re.sub(' +', ' ', x))
text_col = text_col.apply(lambda x: x.replace("<br /><br />",""))

In [19]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kaunghtikesoe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
text_col = text_col.apply(
    lambda x: " ".join([ word for word in x.split() if word not in sw_nltk]) )

In [10]:
df.sample(3)

,review,sentiment
19475,"I really liked this film. All three stars(Connery,Fishburne and Underwood) give credible performances;and Harris is enjoyably over the top. The lighting and shot angles in some of Harris' scenes make his face look truly diabolical. The surprising turn of plot at the end makes it interesting. Not a great movie, but an enjoyable one. I gave it 7 of 10.",1
21282,"Bill Maher's Religulous is not an attack on organized religion. It's an attack on Christianity and Islam. Apart from ridiculing a bunch of Rabbis inventing warped machines to get around Sabbath regulations, he really doesn't attack Judaism and seems enraged when a Rabbi actually challenges the existence of the State of Israel. If Bill Maher followed his hypothesis to its logical conclusion, he would realize that the very creation of Israel in the Palestinian Territories is based on the so called 'holy books' of organized religion. This is evidence of his complete and utter lack of objectivity or focus in the creation of this film.<br /><br />I find it really hard to believe that the man is atheist or even all that intelligent. Anyone can go up to a religious person and laugh at them and call them stupid for their beliefs but what do you have to offer them in return? Nowhere does he actually tell them why he thinks they're stupid. What makes him the ""rational"" person in the room? In a way it reflects how he really isn't and in the process ends up looking just as stupid as those people.<br /><br />If you want to watch a good movie/documentary about the actual evils of religion and how religion can actually be detrimental to the human civilization, watch Richard Dawkins' 'Root of All Evil?'. It is a brilliantly researched documentary, clearly outlining what it hopes to achieve and how.<br /><br />Bill Maher's Religulous is not funny, poses no interesting questions nor does it provide any insight on so controversial a topic. It seems to be the rantings and ravings of an old man disgruntled with his Catholic upbringing. I almost feel sorry for him.",0
31844,"Jörg Buttgereit goes a bit too far with his movies and themes at times, even for my taste but his movies are always something special and hard to classify. They are artistically made, with also often deeper meaning to its themes. This movie is a perfect example of his work.<br /><br />It's also really hard to label this movie. It's not really a movie with a story to it, in a sense of having a beginning, middle and end in it. It also doesn't have a main character but instead focuses on 7 different suicides and killings, on 7 different days.<br /><br />All different stories are being told with lots of class, though some of them are of course more 'interesting' and realistic than the others. They are not necessarily connected but yet together they still tell a story. The movie doesn't feel disjointed at all. All different stories have a different feel to it and Buttgereit tells the story without hardly using any words (also typical for his style) but instead lets the images and obvious sensible emotions of the characters tell the entire story. It helps to make this movie an effective one to watch.<br /><br />Again, the production values all aren't too high and this might be something that might scare off some people. It however helps for this particular movie to set the right tone and atmosphere for the entire movie and its dark, disturbing and depressing themes.<br /><br />A Buttgereit movie that I 'enjoyed' watching.<br /><br />7/10",1


In [11]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split


In [12]:
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], random_state=42, test_size=0.2)

In [13]:
X_train.shape

(40000,)

In [14]:
y_train.shape

(40000,)

In [15]:
pipeline = Pipeline([
    ("vec", CountVectorizer()),
    ("svc", SVC(gamma ='auto'))
])



In [16]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vec', CountVectorizer()), ('svc', SVC(gamma='auto'))])

In [17]:
y_pred = pipeline.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [18]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy_score(y_test, y_pred)

0.7551

In [19]:
confusion_matrix(y_test, y_pred)

array([[3590, 1444],
       [1005, 3961]], dtype=int64)

In [20]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.71      0.75      5034
           1       0.73      0.80      0.76      4966

    accuracy                           0.76     10000
   macro avg       0.76      0.76      0.75     10000
weighted avg       0.76      0.76      0.75     10000



In [21]:

from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], random_state=42, test_size=0.2)

In [23]:
tfidf_vec=TfidfVectorizer(min_df= 10, ngram_range=(1,3))

In [24]:
X_train_bow = tfidf_vec.fit_transform(X_train)
X_test_bow = tfidf_vec.transform(X_test)

In [25]:
svc= svm.SVC(kernel='linear')

In [27]:
svc.fit(X_train_bow, y_train)

SVC(kernel='linear')

In [29]:
y_pred = svc.predict(X_test_bow)
y_pred

array([0, 0, 1, ..., 0, 1, 0], dtype=int64)

In [30]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy_score(y_test, y_pred)

0.9164

In [31]:
confusion_matrix(y_test, y_pred)

array([[4558,  476],
       [ 360, 4606]], dtype=int64)

In [32]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      5034
           1       0.91      0.93      0.92      4966

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000



In [33]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train_bow, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [34]:
y_pred = xgb.predict(X_test_bow)
y_pred

array([0, 0, 1, ..., 0, 1, 0])

In [35]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy_score(y_test, y_pred)

0.8647

In [36]:
confusion_matrix(y_test, y_pred)

array([[4265,  769],
       [ 584, 4382]], dtype=int64)

In [37]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.85      0.86      5034
           1       0.85      0.88      0.87      4966

    accuracy                           0.86     10000
   macro avg       0.87      0.86      0.86     10000
weighted avg       0.87      0.86      0.86     10000

